https://github.com/langchain-ai/rag-from-scratch/blob/main/rag_from_scratch_12_to_14.ipynb

some deviations from the source code because i dont wanna pay for embeddings from openai, or hit openai models. All openAI integration is replaced with ollama.

I also removed langsmith integration. don't think it's needed. just a frontend for LLM debugging which i can achieve with `langchain.debug = True`

In [2]:
import langchain 
langchain.debug = True 

In [1]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

loader = WebBaseLoader("https://lilianweng.github.io/posts/2024-02-05-human-data-quality/")
docs.extend(loader.load())

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
from langchain_ollama.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

# Get llm
llm = ChatOllama(model="llama3.2:3b-instruct-q5_K_M", temperature=0)

# Summarization prompt
summarize_prompt = """Summarize the following document:\n{doc}"""
summarize_prompt_template = ChatPromptTemplate.from_template(summarize_prompt)

# Batch summarize
summarization_chain = (
    {'doc': lambda x: x.page_content}
    | summarize_prompt_template
    | llm
    | StrOutputParser()
)

summaries = summarization_chain.batch(docs, {'max_concurrency': 5})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
[inputs]
[chain/start] [chain:RunnableSequence] Entering Chain run with input:
[inputs]
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<doc>] Entering Chain run with input:
[inputs]
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<doc>] Entering Chain run with input:
[inputs]
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<doc> > chain:RunnableLambda] Entering Chain run with input:
[inputs]
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<doc> > chain:RunnableLambda] s] Exiting Chain run with output:
{
  "output": "\n\n\n\n\n\nLLM Powered Autonomous Agents | Lil'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLil'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n|\n\n\n\n\n\n\nPosts\n\n\n\n\nArchive\n\n\n\n\nSearch\n\n\n\n\nTags\n\n\n\n\nFAQ\n\n\n\n\nemojisearch.app\n\n\n\n\n\n\n\n\n\n      LLM Powered Autonomous Agents\n    \nDate: June 

In [7]:
# Store the summarized doc embedding to full doc pairs
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain.storage import InMemoryByteStore
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_core.documents import Document
import uuid

# Get embedding ollama model
embed = OllamaEmbeddings(
    model="nomic-embed-text"
)

# The vectorstore to use to index the child chunks
vectorstore = Chroma(collection_name="summaries",
                     embedding_function=embed)

# The storage layer for the parent documents
store = InMemoryByteStore()
id_key = "doc_id"

# The retriever
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)
doc_ids = [str(uuid.uuid4()) for _ in docs]

# Docs linked to summaries
summary_docs = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(summaries)
]

# Add
retriever.vectorstore.add_documents(summary_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [8]:
# Pull the summary
query = "Memory in agents"
sub_docs = vectorstore.similarity_search(query,k=1)
sub_docs[0]

Document(id='682d13db-639a-4a6c-9a23-3542667766d4', metadata={'doc_id': '1a26fab4-f019-4b93-aaef-5b58973270a5'}, page_content='Based on the provided text, I will attempt to summarize the main points and identify potential areas for improvement.\n\n**Summary**\n\nThe text appears to be a blog post or research paper discussing the development of autonomous agents powered by large language models (LLMs). The author highlights several challenges and limitations of current LLM-based agent systems, including:\n\n1. Reliability of natural language interface\n2. Robustness to unexpected errors\n3. Limited steerability and control\n\nThe author also mentions various research papers and publications that have addressed these issues, such as "Chain of thought prompting elicits reasoning in large language models" and "ReAct: Synergizing reasoning and acting in language models".\n\n**Potential areas for improvement**\n\n1. **Clarity and organization**: The text is dense and contains multiple sectio

In [16]:
# THen use the summary to pull the full doc
retrieved_docs = retriever.get_relevant_documents(query,n_results=1)
print(retrieved_docs[0].page_content[600:2000])

n

Proof-of-Concept Examples


Challenges

Citation

References





Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:

Planning

Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.
Reflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.


Memory

Short-term memory: I would consider all the in-context learning (See Prompt 

In [ ]:
# Build it all into 1 chain